# Component guide

In [ ]:
import numpy as np
import scipp as sc
import tof

AA = sc.Unit('angstrom')
ms = sc.Unit('ms')
meter = sc.Unit('m')
Hz = sc.Unit('Hz')
deg = sc.Unit('deg')

We begin by making a pulse using the profile from ESS.

In [ ]:
pulse = tof.Pulse.from_facility('ess')
pulse.plot()

In [ ]:
detector = tof.Detector(distance=30. * meter)
model = tof.Model(pulse=pulse, detectors=[detector])
model

In [ ]:
model.run()
model.plot()

As expected, the detector sees all the neutrons from the pulse.

In [ ]:
detector.plot()

In [ ]:
chopper1 = tof.Chopper(
        frequency=40. * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[60., 150.],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[90.0, 190.0],
            unit='deg',
        ),
        phase=30 * deg,
        distance=8 * meter,
        name="Chopper1",
    )
chopper1

In [ ]:
model.choppers = [chopper1]
model.run()
model.plot()

We see that the chopper sees all the incoming neutrons, and blocks many of them (gray) to only let though the ones passing through the openings (blue):

In [ ]:
chopper1.tofs.plot()

In [ ]:
detector.tofs.plot()

In [ ]:
chopper2 = tof.Chopper(
        frequency=20. * Hz,
        open=sc.array(
            dims=['cutout'],
            values=[60., 100., 160., 220.],
            unit='deg',
        ),
        close=sc.array(
            dims=['cutout'],
            values=[90.0, 140.0, 200., 260.],
            unit='deg',
        ),
        phase=60 * deg,
        distance=20 * meter,
        name="Chopper2",
    )

model.choppers = [chopper1, chopper2]
model.run()
model.plot()

In [ ]:
chopper2.plot()

In [ ]:
detector.plot()

In [ ]:
monitor = tof.Detector(distance=15.0 * meter, name='monitor')

model.detectors = [monitor, detector]
model.run()
model.plot()

In [ ]:
monitor.plot()